Part One

In [1]:
import numpy as np
import galois
from itertools import product

In [49]:
with open("input10.txt", "r") as f:
    manual = f.readlines()

targets, all_buttons = [line.split()[0].strip('[]') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [tuple(0 if ch=="." else 1 for ch in t) for t in targets], [[tuple(map(int,v.strip("()").split(","))) for v in lst] for lst in all_buttons]
all_buttons = [[tuple(1 if i in v else 0 for i in range(len(k))) for v in lst] for k,lst in zip(targets,all_buttons)]

sum_min_presses = 0
for inst,(target, buttons) in enumerate(zip(targets,all_buttons)):
    print(f"instance {inst}", end="\r")
    target, buttons = np.array(target).reshape(-1,1), np.array(buttons)
    matrix = galois.GF(2)(np.hstack((buttons.T, target)))
    RREF = matrix.row_reduce()

    #from RREF, gets a dictionary with the possible values of the independent variables and
    #and another with the dependence relations of the dependent ones
    independent_vars, dependent_vars = {}, {}
    for row in RREF[::-1]:
        if (row == 0).all(): continue
        whereone = np.argwhere(row[:-1])
        if whereone.size==1: independent_vars[int(whereone[0,0])]={int(row[-1])}; continue
        for j in whereone[1:].reshape(-1,): independent_vars[int(j)] = {0,1}
        dependent_vars[int(whereone[0,0])] = (tuple(int(i) for i in whereone[1:].reshape(-1,)),int(row[-1]))

    #first an iterable only with tuples filled with the independent vars (elsewhere placeholders)
    pre_possibles = product(*[{j for j in independent_vars[i]} if i in independent_vars else {-1} for i in range(RREF.shape[1]-1)])
    #now get all possible tuples and try the min_presses
    min_presses = np.inf
    for poss in pre_possibles:
        poss_lst = list(poss)
        for ii in dependent_vars:
            poss_lst[ii] = (sum(poss[j] for j in dependent_vars[ii][0])+dependent_vars[ii][1]) %2
        min_presses = min(min_presses, sum(poss_lst))
    # del pre_possibles
    sum_min_presses+=min_presses
print(sum_min_presses,"                    ")

447                     


Around 2.8s cutting off the search early

Tried a more brute-force method... Works very fast with a bit-comparison method if well optimized (i.e. dont use lists when sets do it!). Fraction of a second

In [48]:
from time import time

with open("input10.txt", "r") as f:
    manual = f.readlines()

t0 = time()
targets, all_buttons = [line.split()[0].strip('[]') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [sum((ch=="#")*2**i for i,ch in enumerate(t)) for t in targets], [{tuple(map(int,v.strip("()").split(","))) for v in lst} for lst in all_buttons]
all_buttons = [{sum(2**n for n in b) for b in button} for button in all_buttons]

#as a function called in each instance
def get_n_presses_p1(target :int, buttons :set[int]):
    configs, n_presses = {0}, 0
    while target not in configs:
        configs = {c^b for c in configs for b in buttons}
        n_presses+=1
    return n_presses

sum_min_presses = 0
for target, buttons in zip(targets,all_buttons):
    sum_min_presses+=get_n_presses_p1(target, buttons)

print(f"Total of {sum_min_presses} presses, took {time()-t0}s")

Total of 447 presses, took 0.060202598571777344s


In [56]:
#Sven's
from itertools import product
from time import time

input_file = "input10.txt"

t0 = time()
total_presses = 0
with open(input_file, "r") as file:
    for line in file:
        lights_str, *buttons_str, _ =line.strip().split(" ")

        final_config = sum(2**i*(light=="#") for i, light in enumerate(lights_str[1:-1]))
        
        buttons: set[int] = set()
        for button_str in buttons_str:
            buttons.add(sum(2**int(i) for i in button_str[1:-1].split(",")))

        presses = 0
        lights_configs = {0}
        while final_config not in lights_configs:
            presses += 1

            new_lights_configs = set()
            for lights_config, button in product(lights_configs, buttons):
                new_lights_config = lights_config ^ button
                new_lights_configs.add(new_lights_config)
                if new_lights_config == final_config:
                    break
                
            lights_configs = new_lights_configs
        total_presses += presses

print(f"Part 1: {total_presses}, took {time()-t0}s")

Part 1: 447, took 0.10789155960083008s


Part Two

Now without galois GF(2) fields. Also with allowed ranges for the independent variables

In [2]:
import numpy as np
from itertools import product
from sympy import Matrix

In [3]:
with open("input10.txt", "r") as f:
    manual = f.readlines()
# manual = manual[8:9] #for testing
targets, all_buttons = [line.split('{')[1].strip('}\n') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [tuple(int(i) for i in t.split(',')) for t in targets], [[tuple(map(int,v.strip("()").split(","))) for v in lst] for lst in all_buttons]
all_buttons = [[tuple(1 if i in v else 0 for i in range(len(k))) for v in lst] for k,lst in zip(targets,all_buttons)]

sum_min_presses = 0
for inst,(target, buttons) in enumerate(zip(targets,all_buttons)):
    print(f"instance {inst}", end="\r")

    target, buttons = Matrix(target), Matrix(buttons)
    matrix = Matrix.hstack(buttons.T, target)
    RREF = matrix.rref()[0]
    # display(matrix.hstack(matrix,RREF))
    
    #from RREF, gets a dictionary with the possible values of the independent variables and
    #and another with the dependence relations of the dependent ones
    independent_vars, dependent_vars = {}, {}
    for ii in range(RREF.shape[0]):
        row = RREF[ii,:]
        wherenonzero = np.argwhere(row[:-1])
        if wherenonzero.size == 0: continue
        if wherenonzero.size == 1: independent_vars[int(wherenonzero[0,0])]=[int(row[-1])]; continue
        dependent_vars[int(wherenonzero[0,0])] = (row[-1],tuple((-row[i],int(i)) for i in wherenonzero[1:].reshape(-1,)))
        for j in wherenonzero[1:].reshape(-1,):
            if int(j) not in independent_vars: independent_vars[int(j)] = [0,np.inf]
        if wherenonzero.size==2:
            if row[wherenonzero[1,0]]>0:
                upper_bound = row[-1]//row[wherenonzero[1,0]]
                independent_vars[wherenonzero[1,0]][1] = min(upper_bound, independent_vars[wherenonzero[1,0]][1])
            elif row[-1]<0:
                lower_bound = -(row[-1]//row[wherenonzero[1,0]])
                independent_vars[wherenonzero[1,0]][0] = max(lower_bound, independent_vars[wherenonzero[1,0]][0])

    #tightening bounds on independent variables, essential to reduce from 2h to 25s
    #bounding the inf upper bounds
    unbounded = {i for i in independent_vars if independent_vars[i][-1]==np.inf}
    # print(unbounded)
    for ii in unbounded:
        independent_vars[ii][-1] = min({matrix[row,-1] for row in range(matrix.shape[0]) if matrix[row,ii]})
    # print(independent_vars)
    #tightening from dependent-variable equations (running once is good enough, no need to run while changes)
    for lone_term,dependencies in dependent_vars.values():
        for coeff, var in dependencies:
            #same expression for upper and lower bounds, even though different signs and rounding
            bound = (lone_term + sum(c*independent_vars[v][(-np.sign(c)-1)//2] for c,v in dependencies if v!=var) )//-coeff
            if coeff<0: independent_vars[var][1] = min(bound,independent_vars[var][1])
            else: independent_vars[var][0] = max(bound,independent_vars[var][0]) #coeff>0
    # print(independent_vars)
    #now range format
    independent_vars_ranged = {k:range(v[0],v[-1]+1) for k,v in independent_vars.items()}

    #first an iterable only with tuples filled with the independent vars (elsewhere placeholders)
    pre_possibles = set(product(*[{j for j in independent_vars_ranged[i]} if i in independent_vars_ranged else {np.nan} for i in range(RREF.shape[1]-1)]))
    #now populate with all possible tuples
    min_presses = np.inf
    subtracted_row = np.array([1]*RREF.shape[1]) - np.sum(RREF, axis=0) #last element is actually (min_presses - np.sum(RREF[:,-1])), but we don't have a meaningful min_presses yet
    while pre_possibles:
        poss_lst = list(pre_possibles.pop())
        for i in dependent_vars:
            poss_lst[i] = dependent_vars[i][0] + sum(coeff*poss_lst[idx] for coeff,idx in dependent_vars[i][1])
        if all(v//1==v and v >= 0 for v in poss_lst):
            if sum(poss_lst) < min_presses:
                min_presses = sum(poss_lst) #min(min_presses, sum(poss_lst))
                #reducing range of independent variables
                for i in np.argwhere(subtracted_row[:-1]).reshape(-1,):
                    # lhs of subtracted_row = min_presses - np.sum(RREF[:,-1])
                    bound = (min_presses - np.sum(RREF[:,-1]) - sum(subtracted_row[ii]*(independent_vars[ii][0 if subtracted_row[ii]>0 else -1]) for ii in independent_vars if i!=ii))//subtracted_row[i]
                    # print(bound)
                    if subtracted_row[i]>0: independent_vars[i][-1] = min(independent_vars[i][-1], bound)
                    else:                   independent_vars[i][0]  = max(independent_vars[i][0], bound)  #subtracted_row[i]<0
                pre_possibles.difference_update({poss for poss in pre_possibles if True in {poss[ii]>independent_vars[ii][-1] for ii in independent_vars}})
            #     print(independent_vars)
            # print(f"presses={poss_lst}, sum={sum(poss_lst)}, min_presses={min_presses}")
    # print(min_presses)
    sum_min_presses += min_presses

print("Final sum is", sum_min_presses)

Final sum is 18960


With each feasible result, a reduction of the ranges of the independent variables is applied. This reduces the time to below 20s.

#### The reddit technique of using recursion and the parity function

The method takes the target, calculates how to get target mod 2 usign the method from part 1, then the rest, noticing that it is an all-even target, is replaced by taking twice the buttons calculated on a halved target. The latter is done with recursion.

To make it work, one has to consider all possibilities for the parity. Also notice that the recursiveness has to take into account that some buttons were pressed twice (or more), so some results may not be taken mod 2.

In [51]:
import galois

cache_par = {}
def get_parity_presses(buttons :np.ndarray, target :np.ndarray):
    if tuple(target[0]) in cache_par: return cache_par[tuple(target[0])]
    matrix = galois.GF(2)(np.hstack((buttons.T, target.T)))
    RREF = matrix.row_reduce()

    #from RREF, gets a dictionary with the possible values of the independent variables and
    #and another with the dependence relations of the dependent ones
    independent_vars, dependent_vars = {}, {}
    for row in RREF[::-1]:
        if (row == 0).all(): continue
        whereone = np.argwhere(row[:-1])
        if whereone.size==1: independent_vars[int(whereone[0,0])]={int(row[-1])}; continue
        for j in whereone[1:].reshape(-1,): independent_vars[int(j)] = {0,1}
        try:
            dependent_vars[int(whereone[0,0])] = (tuple(int(i) for i in whereone[1:].reshape(-1,)),int(row[-1]))
        except IndexError: return 10**3*np.ones((1,buttons.shape[0]), dtype=int) #print(RREF, "infeasible matrix"); 

    #first an iterable only with tuples filled with the independent vars (elsewhere placeholders)
    pre_possibles = list(product(*[{j for j in independent_vars[i]} if i in independent_vars else {-1} for i in range(RREF.shape[1]-1)]))
    #now get all possible tuples and get all possible combinations (not just min)
    for i in range(len(pre_possibles)):
        poss_lst = list(pre_possibles[i])
        for ii in dependent_vars:
            poss_lst[ii] = (sum(poss_lst[j] for j in dependent_vars[ii][0])+dependent_vars[ii][1]) %2
        pre_possibles[i] = poss_lst
    out = np.array(pre_possibles)
    cache_par[tuple(target[0])] = out
    return out

cache_n = {}
def get_n_presses(buttons, target):
    if tuple(target[0]) in cache_n: return cache_n[tuple(target[0])]
    # print("target\n", target)
    n_par_presses = get_parity_presses(buttons, target%2)
    if (target<=1).all():
        idx = np.argmin(np.sum(n_par_presses, axis=1))
        return n_par_presses[idx,:]
    # print("n_par_presses\n",n_par_presses)
    new_targets    = (target-n_par_presses@buttons)//2
    n_par_presses  = np.array([n_par_presses[i] for i in range(n_par_presses.shape[0]) if (new_targets[i]>=0).all()])
    if n_par_presses.size==0:
        out = 10**3*np.ones((buttons.shape[0]), dtype=int)
        cache_n[tuple(target[0])] = out
        return out
    new_targets = np.array([row for row in new_targets if (row>=0).all()])
    # print("new targets\n", new_targets)
    n_presses = np.array([get_n_presses(buttons, tgt.reshape(1,-1)) for tgt in new_targets])
    # print("n_presses\n",n_presses)
    idx = np.argmin(np.sum(n_par_presses + 2* n_presses, axis=1))
    out = n_par_presses[idx,:] + 2*n_presses[idx,:]
    cache_n[tuple(target[0])] = out
    return out

with open("input10.txt", "r") as f:
    manual = f.readlines()
# manual = manual[92:93] #for testing

targets, all_buttons = [line.split('{')[1].strip('}\n') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [tuple(int(i) for i in t.split(',')) for t in targets], [[tuple(map(int,v.strip("()").split(","))) for v in lst] for lst in all_buttons]
all_buttons = [[tuple(1 if i in v else 0 for i in range(len(k))) for v in lst] for k,lst in zip(targets,all_buttons)]

# get_parity_presses(np.array(all_buttons[0]), np.array(targets[0]).reshape(1,-1)%2)
sum_min_presses = 0
for inst,(target, buttons) in enumerate(zip(targets,all_buttons)):
    print(f"instance {inst}", end="\r")
    cache_par, cache_n = {}, {}
    min_presses = np.sum(get_n_presses(np.array(buttons).astype(np.int32), np.array(target).reshape(1,-1).astype(np.int32)))
    # print(min_presses)
    sum_min_presses+= min_presses
sum_min_presses

np.int64(18960)

The method works, but still slow compared to the linear-algebra one. Let's use it with a faster part-1 method

In [8]:
from time import time
from collections import defaultdict

with open("input10.txt", "r") as f:
    manual = f.readlines()

t0 = time()
targets, all_buttons = [line.split('{')[1].strip('}\n') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [list(map(int,t.split(","))) for t in targets], [{tuple(map(int,v.strip("()").split(","))) for v in lst} for lst in all_buttons]
all_buttons = [{sum(2**n for n in b) for b in button} for button in all_buttons]


# a faster parity check function
cache_p = {}
def get_presses_p1(target :int, buttons :set[int]):
    if target in cache_p: return cache_p[target]
    configs, out_configs = defaultdict(set), set()
    configs[0] = {()}
    if target==0: out_configs.add(())
    # configs, out_configs = {0:[]}, set()
    for _ in range(len(buttons)):
        # configs = {c^b: history+[b] for c,history in configs.items() for b in buttons if b not in history}
        # if target in configs:
        #     out_configs.add(tuple(configs[target]))
        new_configs = defaultdict(set)
        for b in buttons:
            for c,h_set in configs.items():
                for history in h_set:
                    if b not in history:
                        new_configs[c^b].add(tuple(sorted(history+(b,))))
        configs = new_configs
        if target in new_configs:
            for history in new_configs[target]: out_configs.add(tuple(history))
    cache_p[target] = out_configs
    return out_configs

# cache_p = {}
# def get_presses_p1(target :int, buttons :set[int]):
#     if target in cache_p: return cache_p[target]
#     configs, out_configs = {(0,0):()}, set()
#     # if target==0: out_configs.add(())
#     # configs, out_configs = {0:[]}, set()
#     for _ in range(len(buttons)):
#         configs = {(c^b,b): (*history,b) for (c,_),history in configs.items() for b in buttons if b not in history}
#         for b in buttons:
#             if (target,b) in configs: out_configs.add(tuple(sorted(configs[target,b])))

#     cache_p[target] = out_configs
#     return out_configs

# cache_p = {}
# def get_presses_p1(target :int, buttons :set[int]):
#     if target in cache_p: return cache_p[target]
#     configs, new_configs = defaultdict(set), defaultdict(set)
#     configs[0], out_configs = {()}, set()
#     # if target==0: out_configs.add(())
#     for _ in range(len(buttons)):
#         for c, history in configs.items():
#             for b in buttons:
#                 for h in history:
#                     if b not in h:
#                         new_configs[c^b].add(tuple(sorted(h + (b,))))
#         configs.update(new_configs)
#         # print(configs)
#     out_configs = configs[target]
#     cache_p[target] = out_configs
#     return out_configs

cache_n = {}
def get_n_presses(target, buttons):
    if tuple(target) in cache_n:
        # print("cached (n)", target)
        return cache_n[tuple(target)]
    p1_target = sum(2**i*(t%2) for i,t in enumerate(target))
    p1_sol = get_presses_p1(p1_target, buttons)
    # print(p1_target, p1_sol)
    if not p1_sol: return 10**4
    if all(t_i<=1 for t_i in target):
        cache_n[tuple(target)] = len(min(p1_sol, key=len))
        return len(min(p1_sol, key=len))
    n_out = 10**4
    for p1_buttons in p1_sol:
        mod_target = target.copy()
        for b in p1_buttons:
            for i in range(1,len(bin(b))-1):
                mod_target[i-1]-=int(bin(b)[-i])
        # print(mod_target, end=" ")
        if any(n<0 for n in mod_target): continue
        mod_target = list(map(lambda x:x//2, mod_target))
        # print(mod_target)
        n_out = min(n_out, len(p1_buttons) + 2* get_n_presses(mod_target, buttons))
    cache_n[tuple(target)] = n_out
    return n_out
    
sum_min_presses = 0
for i, (target, buttons) in enumerate(zip(targets[0:3],all_buttons[0:3])):
    cache_p, cache_n = {}, {}
    print(f"instance {i}, target {target}", end="\r")
    min_presses = get_n_presses(target, buttons)
    # print(min_presses)
    sum_min_presses += min_presses

sum_min_presses, time()-t0

(345, 47.814696311950684)

In [ ]:
from collections import defaultdict

with open("input10.txt", "r") as f:
    manual = f.readlines()

targets, all_buttons = [line.split('{')[1].strip('}\n') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [list(map(int,t.split(","))) for t in targets], [{tuple(map(int,v.strip("()").split(","))) for v in lst} for lst in all_buttons]
all_buttons = [{sum(2**n for n in b) for b in button} for button in all_buttons]

cache_p = {}
def get_presses_p1(target :int, buttons :set[int]):
    if target in cache_p: return cache_p[target]
    configs, new_configs = defaultdict(set), defaultdict(set)
    configs[0], out_configs = {()}, set()
    # if target==0: out_configs.add(())
    # for b in buttons:
    #I DON'T HAVE A GOOD IDEA HERE!

    for _ in range(len(buttons)):
        for c, history in configs.items():
            for b in buttons:
                for h in history:
                    if b not in h:
                        new_configs[c^b].add(tuple(sorted(h + (b,))))
        configs.update(new_configs)
        # print(configs)
    out_configs = configs[target]

    cache_p[target] = out_configs
    return out_configs

target, buttons = targets[0], all_buttons[0]
p1_target = sum(2**i*(t%2) for i,t in enumerate(target))
get_presses_p1(p1_target, buttons)

{(35, 272, 291, 526, 987, 1018), (152, 550, 858, 987)}

This method uses the ideas of the fast parity checker, but changes had to made to get all combinations, and that made it heavy and slow. It ends up being very slow, 4min30

#### Feels like cheating: LP solvers + branch-and-bound

In [3]:
import numpy as np
import cvxpy as cp

In [4]:
with open("input10.txt", "r") as f:
    manual = f.readlines()
# manual = manual[:3] #for testing
targets, all_buttons = [line.split('{')[1].strip('}\n') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [tuple(int(i) for i in t.split(',')) for t in targets], [[tuple(map(int,v.strip("()").split(","))) for v in lst] for lst in all_buttons]
all_buttons = [[tuple(1 if i in v else 0 for i in range(len(k))) for v in lst] for k,lst in zip(targets,all_buttons)]

EPS = .00001

sum_min_presses = 0
for inst,(target, buttons) in enumerate(zip(targets,all_buttons)):
    print(f"target {inst}:", target, end="                \r")

    target, buttons = np.array(target), np.array(buttons).T
    x = cp.Variable(buttons.shape[1])
    # buttons_cp = cp.Parameter(buttons.shape); buttons_cp.value = buttons
    objective = cp.Minimize(cp.sum(x))
    constraints = [buttons@x == target, x>=0]
    problem = cp.Problem(objective, constraints)
    problem.solve("SCIPY")
    if problem.status == 'optimal' and np.allclose(x.value, np.round(x.value)): sum_min_presses+=int(np.round(problem.value)); continue
    #else:branch
    non_int_idx = np.argwhere(abs(x.value-np.round(x.value))>EPS)[0]
    int_down, int_up = int(x.value[non_int_idx][0]), int(x.value[non_int_idx][0])+1
    branches = [(problem.value, [x[non_int_idx] <= int_down], x.value), (problem.value, [x[non_int_idx] >= int_up], x.value)]
    upper_bound = np.inf
    while branches:
        _, branch_constraints, __ = min(branches, key=lambda br:br[0])
        branches.remove((_,branch_constraints,__))
        problem = cp.Problem(objective, constraints+branch_constraints)
        problem.solve("SCIPY")
        if problem.status == 'optimal' and np.allclose(x.value, np.round(x.value)):
            upper_bound = min(problem.value, upper_bound)
            branches = [br for br in branches if br[0]<upper_bound]
        else:
            if problem.status == 'infeasible': continue
            non_int_idx = np.argwhere(abs(x.value-np.round(x.value))>EPS)[0]
            int_down, int_up = int(x.value[non_int_idx][0]), int(x.value[non_int_idx][0])+1
            branches.append((problem.value, branch_constraints+[x[non_int_idx] <= int_down], x.value))
            branches.append((problem.value, branch_constraints+[x[non_int_idx] >= int_up], x.value))
    sum_min_presses+=int(np.round(upper_bound))

sum_min_presses

18960

Around 7s. But it feels like cheating...